In [2]:
@file:DependsOn("io.ktor:ktor-client-core:3.1.3")
@file:DependsOn("io.ktor:ktor-client-cio:3.1.3")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:3.1.3")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:3.1.3")
@file:DependsOn("com.charleskorn.kaml:kaml-jvm:0.67.0")
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe

import ro.jf.funds.client.notebook.*

val client = FundsClient()
val user = client.ensureUserExists("Johann-18.3")
user

UserTO(id=7069fa18-f028-4b65-b378-31a0266d611b, username=Johann-18.3)

In [3]:
import ro.jf.funds.account.sdk.AccountSdk
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk

val ACCOUNTS_YAML_FILE = "../../data/provision/accounts.yaml"
val FUNDS_YAML_FILE = "../../data/provision/funds.yaml"
val IMPORT_CONFIGURATION_JSON_FILE = "../../data/provision/import-configuration.json"
val IMPORT_CONFIGURATION_YAML_FILE = "../../data/provision/import-configuration.yaml"

val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()


In [4]:
import ro.jf.funds.account.api.model.*
import ro.jf.funds.commons.model.*
import kotlinx.coroutines.runBlocking
import com.charleskorn.kaml.Yaml
import kotlinx.serialization.builtins.ListSerializer

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(user.id).items
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = File(ACCOUNTS_YAML_FILE).readText()
        .let { Yaml.default.decodeFromString(ListSerializer(CreateAccountTO.serializer()), it) }
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(user.id, it) }
    existingAccounts + newAccounts
}
accounts.map { it.toString() }.joinToString(separator = "\n")

AccountTO(id=94b95af0-486c-4a27-bf40-ef1031291621, name=Cash RON, unit=Currency(value=RON))
AccountTO(id=164636d5-8f21-4b9a-a470-4d61c6399651, name=Cash EUR, unit=Currency(value=EUR))
AccountTO(id=80231cc0-e075-40df-ad05-846ceb0008e0, name=ING RON, unit=Currency(value=RON))
AccountTO(id=22bcf473-349e-4759-8286-9367a9474c54, name=ING Economy, unit=Currency(value=RON))
AccountTO(id=62195944-9922-4c12-9fab-400ee0d4d253, name=Food Coupons, unit=Currency(value=RON))
AccountTO(id=0bd5d2ac-04bd-4ca2-9614-415bc0bc77bc, name=Revolut RON, unit=Currency(value=RON))
AccountTO(id=b50caf21-d60a-466d-873e-fba3b6fdf5e0, name=Other, unit=Currency(value=RON))
AccountTO(id=808a32d8-7b37-4580-96fa-ab676463378a, name=BT RON, unit=Currency(value=RON))
AccountTO(id=192cca1a-8673-4d83-af24-60fbe9e66423, name=BT Economy RON, unit=Currency(value=RON))

In [5]:
import ro.jf.funds.fund.api.model.*
import kotlinx.coroutines.runBlocking

val funds = runBlocking {
    val existingFunds = fundSdk.listFunds(user.id).items
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = File(FUNDS_YAML_FILE).readText()
        .let { Yaml.default.decodeFromString(ListSerializer(CreateFundTO.serializer()), it) }
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(user.id, it) }
    existingFunds + newFunds
}
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=429a4bb1-512c-419b-8a04-2da54d4717a6, name=Expenses)
FundTO(id=627bde37-5fa9-4364-a431-2cd147a28ecb, name=Work Income)
FundTO(id=ea3e5b72-5095-4711-aa78-a4f5c720f6d9, name=Savings)
FundTO(id=308d7284-e713-4c4a-b777-b40f82cf2a13, name=Gifts)

In [6]:
import ro.jf.funds.importer.api.model.*
import kotlinx.serialization.json.Json

val dataConfigurationRawYaml: String = File(IMPORT_CONFIGURATION_YAML_FILE).readText()
val importConfiguration = Yaml.default.decodeFromString(ImportConfigurationTO.serializer(), dataConfigurationRawYaml)

In [7]:
import kotlinx.coroutines.delay
import kotlinx.coroutines.runBlocking
import kotlinx.datetime.Clock
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*
import kotlin.time.Duration

val csvFiles =
    listOf(File("../../data/wallet/2019"), File("../../data/wallet/2020"))
        .flatMap { it.listFiles()?.toList() ?: error("no files found") }
runBlocking {
    var importTask = importSdk.import(user.id, importConfiguration, csvFiles)
    val now: Instant = Clock.System.now()
    val timeout = 60.seconds
    while (importTask.status == ImportTaskTO.Status.IN_PROGRESS && Clock.System.now() - now < timeout) {
        delay(500)
        importTask = importSdk.getImportTask(user.id, importTask.taskId)
    }
    importTask
}


ImportTaskTO(taskId=d756a5ee-9f68-483e-8fc8-116e7c607345, status=COMPLETED, reason=null)